In [21]:
import os
import json
import sqlite3
import requests
from pprint import pprint
from datetime import datetime, timedelta, UTC
from typing import TypedDict, Optional, Dict, Any
from config import DB_PATH, CRYPTO_API_TOKEN, API_BASE_URL, headers

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferWindowMemory
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4.7-flash",
    openai_api_key=os.getenv("ZAI_API_KEY"),
    openai_api_base="https://api.z.ai/api/paas/v4/"
)

In [4]:
llm.invoke("Hello, how are you?").content

"Hello! I'm doing well, thank you for asking. How can I help you today?"

In [5]:
STALE_MINUTES = 5
REJECTION_MESSAGE = "INSUFFICIENT DATA – Not found in Knowledge Base or API"
DISALLOWED_INTENTS = ["prediction", "advice", "hypothetical"]

In [22]:
def call_crypto_api(symbol: str) -> Optional[Dict[str, Any]]:
    """
    Calls FreeCryptoAPI /getData endpoint and extracts relevant fields.
    
    Returns a flat dict with the fields we care about + our fetch timestamp.
    Returns None on failure or invalid response.
    
    Note: This function no longer takes 'data_type' because the endpoint
    returns all relevant price fields at once.
    """
    url = f"{API_BASE_URL}getData?symbol={symbol.upper()}"
    
    try:
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        data = response.json()

        if data.get("status") != "success":
            return None

        symbols = data.get("symbols", [])
        if not symbols:
            return None

        # Take first item (we requested one symbol)
        item = symbols[0]

        return {
            "symbol": item.get("symbol"),
            "last_price": float(item.get("last", 0)),
            "lowest_price": float(item.get("lowest", 0)),
            "highest_price": float(item.get("highest", 0)),
            "daily_change_percentage": float(item.get("daily_change_percentage", 0)),
            "date": item.get("date"),
            "source_exchange": item.get("source_exchange"),
            "fetch_timestamp": datetime.now(UTC).isoformat()
        }

    except (requests.RequestException, ValueError, KeyError, TypeError):
        return None

In [23]:
response = call_crypto_api("BTC")

In [25]:
print(json.dumps(response, indent=3))

{
   "symbol": "BTC",
   "last_price": 89615.7,
   "lowest_price": 87932.07,
   "highest_price": 90099.5,
   "daily_change_percentage": -0.09481526074197,
   "date": "2026-01-21 09:55:16",
   "source_exchange": "binance",
   "fetch_timestamp": "2026-01-21T06:55:17.100088+00:00"
}
